# Toy employees database (R/MySQL)

In [2]:
# Libraries
library(tidyverse)
library(odbc)
library(DBI)

Warning message:
"package 'ggplot2' was built under R version 4.5.2"
── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Warning message:
"package 'odbc' was built under R version 4.5.2"
Warning message:
"package 'DBI' was built under R version 4.5.2"


In this notebook, we query a toy schema provided by IBM. The schema has already been created. All therefore need to do is connect to the MySQL database.

## Check R-MySQL data type conversion

In [ ]:
# Data types
menagerie <- c(1, 1L, "1", TRUE, list(raw(1)))

for (animal in menagerie) {
    animal_type <- typeof(animal)

    tryCatch({
        print(
            paste(
                animal_type, ":",
                #dbDataType(RSQLite::SQLite(), animal)
                dbDataType(RMariaDB::MariaDB(), animal)
                #dbDataType(RMySQL::MySQL(), animal)
                #dbDataType(RPostgres::Postgres(), animal)
            )
        )
    }, error = function(e) {
        warning(paste(animal_type, ": unsupported type"))
    })
}

[1] "double : DOUBLE"
[1] "integer : INTEGER"
[1] "character : VARCHAR(1)"
[1] "logical : TINYINT"


Warning message in value[[3L]](cond):
"raw : unsupported type"


## Connection

In [4]:
# List available drivers
odbcListDrivers() |> 
    tibble() |>
    filter(str_detect(name, "SQL"))

name,attribute,value
<chr>,<chr>,<chr>
SQL Server,APILevel,2
SQL Server,ConnectFunctions,YYY
SQL Server,CPTimeout,60
SQL Server,DriverODBCVer,03.50
SQL Server,FileUsage,0
SQL Server,SQLLevel,1
SQL Server,UsageCount,1
MySQL ODBC 9.6 ANSI Driver,UsageCount,1
MySQL ODBC 9.6 Unicode Driver,UsageCount,1


In [5]:

# Establish connection
c <- dbConnect(
	drv = odbc(),
	driver = "MySQL ODBC 9.6 Unicode Driver", # MySQL, PostgreSQL, SQLite3
	database = "mysql", # MySQL mysql, PostgreSQL postgres, SQLite :memory:
	server = "localhost",
	uid = "r_user",
	pwd = "sql_r",
	port = 3306 # MySQL 3306, PostgreSQL 5432, SQLite 5432, SQL Server 1433
)

In [11]:
# Display databases
#c |> dbGetQuery("SHOW DATABASES;")

In [9]:
# Select database
c |> dbGetQuery("USE ibm_employees_02;")

<0 x 0 matrix>

In [ ]:
# Display available tables
q <- "
SHOW TABLES;
"
dbGetQuery(c, q)

Tables_in_ibm_employees_02
<chr>
departments
employees
job_history
jobs
locations


## String patterns, sorting, grouping